<a href="https://colab.research.google.com/github/SelinDenizz/Freudian-Dream-Interpretation-Model/blob/main/notebook/data_trainer_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import importlib.util
import os

def is_installed(package_name):
    return importlib.util.find_spec(package_name) is not None

if not is_installed("unsloth"):

    !pip uninstall -y protobuf fsspec torch torchaudio torchvision
    !pip install "protobuf<4.0.0" fsspec==2025.3.0
    !pip install torch==2.6.0+cu124 torchaudio==2.6.0+cu124 torchvision==0.21.0+cu124 --index-url https://download.pytorch.org/whl/cu124
    !pip install -q unsloth transformers peft datasets gradio pandas tqdm
    !pip check

    requirements_path = "/content/drive/MyDrive/freudian_dream_analyzer/requirements.txt"
    !mkdir -p /content/drive/MyDrive/freudian_dream_analyzer
    !pip freeze > "{requirements_path}"
    print(f"requirements.txt saved at {requirements_path}")
else:
    print("Environment already installed, skipping setup.")

In [ ]:
# Runtime & GPU check
import sys
import torch

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    print("Running in Google Colab")

if torch.cuda.is_available():
    print(f"GPU is available: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
else:
    print("No GPU available. Enable it from Runtime > Change runtime type.")

In [ ]:
# First, unmount if already mounted
from google.colab import drive
try:
    drive.flush_and_unmount()
    print('Existing drive mount was unmounted')
except:
    print('No existing drive mount')

# Check and clean up the mount point
import os
import shutil
if os.path.exists('/content/drive'):
    if os.path.isdir('/content/drive') and os.listdir('/content/drive'):
        shutil.rmtree('/content/drive')
        print("Removed existing /content/drive directory and its contents")

    elif os.path.isfile('/content/drive'):
        os.remove('/content/drive')
        print("Removed existing /content/drive file")

os.makedirs('/content/drive', exist_ok=True)
print("Created fresh /content/drive directory")

drive.mount('/content/drive')

%cd /content/drive/MyDrive/freudian_dream_analyzer/

import sys
sys.path.append('/content/drive/MyDrive/freudian_dream_analyzer/')

In [ ]:
import sys
import importlib.util
import os

project_root = "/content/drive/MyDrive/freudian_dream_analyzer"
sys.path.append(project_root)

module_path = os.path.join(project_root, "script/module/data_trainer.py")

spec = importlib.util.spec_from_file_location("data_trainer", module_path)
data_trainer = importlib.util.module_from_spec(spec)
spec.loader.exec_module(data_trainer)

UnslothTrainer = data_trainer.UnslothTrainer


In [ ]:
trainer = UnslothTrainer(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    max_seq_length=2048,
    micro_batch_size=1,
    gradient_accumulation_steps=4,
    num_epochs=3,
    learning_rate=2e-4,
    lora_r=int(16),
    lora_alpha=32,
    lora_dropout=0.05,
    bf16=True,
    tf32=False,
    save_steps=100
)

model_path = trainer.finetune(
    jsonl_file="/content/drive/MyDrive/freudian_dream_analyzer/data/dream/processed/fine_tuning_format/dreambank_finetune_llama.jsonl",
    output_dir="/content/drive/MyDrive/freudian_dream_analyzer/model/unsloth_model"
)

In [5]:
# =============================================================================
# UPLOAD YOUR CUSTOM MODEL TO GOOGLE COLAB
# Run this in a separate cell BEFORE running the main application
# =============================================================================

import os
from google.colab import files

# Create the model directory
model_dir = '/content/model/unsloth_model'
os.makedirs(model_dir, exist_ok=True)
print(f"✅ Created directory: {model_dir}")

# Upload your model files
print("""
📁 UPLOAD YOUR MODEL FILES:

From your local computer, you need to upload these files from your 'model/unsloth_model' folder:
- adapter_config.json
- adapter_model.safetensors
- special_tokens_map.json
- tokenizer_config.json
- tokenizer.json
- tokenizer.model
- Any other files in your model directory

Click 'Choose Files' below and select ALL your model files:
""")

# File upload interface
uploaded = files.upload()

# Move uploaded files to the correct directory
if uploaded:
    print(f"\n📂 Processing {len(uploaded)} uploaded files:")
    for filename, content in uploaded.items():
        filepath = os.path.join(model_dir, filename)
        with open(filepath, 'wb') as f:
            f.write(content)
        print(f"✅ Saved: {filename} ({len(content)} bytes)")

    print(f"\n🎯 Model files uploaded to: {model_dir}")

    # Verify the upload
    print(f"\n📋 Files in model directory:")
    try:
        for file in sorted(os.listdir(model_dir)):
            file_path = os.path.join(model_dir, file)
            size = os.path.getsize(file_path)
            print(f"  - {file} ({size:,} bytes)")
    except Exception as e:
        print(f"  Error listing files: {e}")

    # Check for required LoRA files
    adapter_config = os.path.join(model_dir, 'adapter_config.json')
    adapter_model = os.path.join(model_dir, 'adapter_model.safetensors')

    if os.path.exists(adapter_config):
        print(f"\n✅ Found adapter_config.json - LoRA model detected")

        # Read adapter config to get base model info
        try:
            import json
            with open(adapter_config, 'r') as f:
                config = json.load(f)
            base_model = config.get('base_model_name_or_path', 'Unknown')
            print(f"📊 Base model: {base_model}")
        except Exception as e:
            print(f"⚠️ Could not read adapter config: {e}")

    if os.path.exists(adapter_model):
        print(f"✅ Found adapter_model.safetensors")
    else:
        print(f"⚠️ adapter_model.safetensors not found - this might cause issues")

    print(f"\n🚀 Ready! Now run the main application script to use your custom model.")

else:
    print("❌ No files uploaded. Please try again.")

# =============================================================================
# ALTERNATIVE: Upload via Google Drive (if you have files there)
# =============================================================================

print("""
📁 ALTERNATIVE UPLOAD METHOD - Google Drive:

If your model files are in Google Drive, run this instead:

```python
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Copy your model files from Drive
# Update this path to where your model files are stored in Drive
source_path = '/content/drive/MyDrive/your_model_folder/unsloth_model'
destination_path = '/content/model/unsloth_model'

if os.path.exists(source_path):
    shutil.copytree(source_path, destination_path, dirs_exist_ok=True)
    print(f"✅ Copied model from Google Drive: {source_path}")

    # List copied files
    for file in os.listdir(destination_path):
        print(f"  - {file}")
else:
    print(f"❌ Model folder not found in Google Drive: {source_path}")
    print("Please update the source_path to match your Drive folder structure")
```
""")

# =============================================================================
# VERIFY MODEL STRUCTURE
# =============================================================================

def verify_model_structure():
    """Verify if the uploaded model has the correct structure"""
    model_dir = '/content/model/unsloth_model'

    if not os.path.exists(model_dir):
        print("❌ Model directory doesn't exist yet")
        return False

    files = os.listdir(model_dir)
    if not files:
        print("❌ Model directory is empty")
        return False

    # Check for LoRA adapter files
    required_lora_files = ['adapter_config.json']
    lora_files_present = [f for f in required_lora_files if f in files]

    # Check for full model files
    common_model_files = ['config.json', 'pytorch_model.bin', 'model.safetensors']
    model_files_present = [f for f in common_model_files if f in files]

    if lora_files_present:
        print("✅ LoRA adapter model detected")
        return True
    elif model_files_present:
        print("✅ Full model detected")
        return True
    else:
        print("⚠️ Unknown model structure")
        print(f"Files found: {files}")
        return True  # Let the app try to load it anyway

# Run verification
print("\n" + "="*50)
print("MODEL VERIFICATION:")
verify_model_structure()

✅ Created directory: /content/model/unsloth_model

📁 UPLOAD YOUR MODEL FILES:

From your local computer, you need to upload these files from your 'model/unsloth_model' folder:
- adapter_config.json
- adapter_model.safetensors  
- special_tokens_map.json
- tokenizer_config.json
- tokenizer.json
- tokenizer.model
- Any other files in your model directory

Click 'Choose Files' below and select ALL your model files:



Saving adapter_config.json to adapter_config.json
Saving adapter_model.safetensors to adapter_model.safetensors
Saving special_tokens_map.json to special_tokens_map.json
Saving tokenizer.json to tokenizer.json
Saving tokenizer.model to tokenizer.model
Saving tokenizer_config.json to tokenizer_config.json

📂 Processing 6 uploaded files:
✅ Saved: adapter_config.json (841 bytes)
✅ Saved: adapter_model.safetensors (159967880 bytes)
✅ Saved: special_tokens_map.json (582 bytes)
✅ Saved: tokenizer.json (3896154 bytes)
✅ Saved: tokenizer.model (499723 bytes)
✅ Saved: tokenizer_config.json (1838 bytes)

🎯 Model files uploaded to: /content/model/unsloth_model

📋 Files in model directory:
  - adapter_config.json (841 bytes)
  - adapter_model.safetensors (159,967,880 bytes)
  - special_tokens_map.json (582 bytes)
  - tokenizer.json (3,896,154 bytes)
  - tokenizer.model (499,723 bytes)
  - tokenizer_config.json (1,838 bytes)

✅ Found adapter_config.json - LoRA model detected
📊 Base model: unsloth/l

True

In [10]:
# =============================================================================
# GET YOUR MODEL WORKING WITH HUGGING FACE AUTHENTICATION
# This will properly load your LoRA on the correct Llama-2 base model
# =============================================================================

print("🔑 SETTING UP HUGGING FACE AUTHENTICATION")
print("=" * 50)

print("""
🎯 YOUR LORA NEEDS LLAMA-2 BASE MODEL!

The issue: Your LoRA was trained on Llama-2 architecture, but we can't access it without authentication.

SOLUTION: Get authenticated access to Llama-2 models

📋 STEP-BY-STEP INSTRUCTIONS:

1. 🌐 Go to: https://huggingface.co/meta-llama/Llama-2-7b-chat-hf
2. 📝 Create a Hugging Face account (if you don't have one)
3. ✅ Request access to Llama-2 (usually approved quickly)
4. 🔑 Create an access token:
   - Go to: https://huggingface.co/settings/tokens
   - Click "New token"
   - Choose "Read" permissions
   - Copy the token

5. 🔐 Add token to Colab:
   - Click the key icon (🔑) in the left sidebar
   - Add new secret: Name = "HF_TOKEN", Value = your token
   - Enable notebook access

OR run this code with your token:
""")

# Provide authentication options
print("🔑 AUTHENTICATION OPTIONS:")
print("\nOption 1 - Add HF_TOKEN to Colab secrets (RECOMMENDED)")
print("Option 2 - Login directly (run the code below)")

from huggingface_hub import login
from google.colab import userdata
import os

# Try to get token from secrets first
try:
    hf_token = userdata.get('HF_TOKEN')
    login(hf_token)
    print("✅ Successfully authenticated with Colab secrets!")
    authenticated = True
except Exception as e:
    print(f"⚠️ No HF_TOKEN in secrets: {e}")
    authenticated = False

if not authenticated:
    print("\n🔑 MANUAL LOGIN:")
    print("Run this code and enter your Hugging Face token when prompted:")
    print("from huggingface_hub import login")
    print("login()")

    # Try interactive login
    try:
        login()
        authenticated = True
        print("✅ Authentication successful!")
    except Exception as e:
        print(f"❌ Authentication failed: {e}")
        authenticated = False

if not authenticated:
    print("""
    ❌ AUTHENTICATION REQUIRED!

    Your LoRA adapter MUST be used with Llama-2 base model.
    Without authentication, we can only use incompatible models.

    Please:
    1. Get Llama-2 access at: https://huggingface.co/meta-llama/Llama-2-7b-chat-hf
    2. Create HF token at: https://huggingface.co/settings/tokens
    3. Add token to Colab secrets as "HF_TOKEN"
    4. Restart and run this script again

    Your model configuration shows it needs:
    - Base: unsloth/llama-2-7b-chat-bnb-4bit
    - Target modules: ['down_proj', 'v_proj', 'q_proj', 'gate_proj', 'k_proj', 'up_proj', 'o_proj']

    These are Llama-2 specific layers that don't exist in other models!
    """)

    # Provide fallback demo
    print("\n🔄 FALLBACK OPTION:")
    print("I can create a DEMO version that simulates your model's responses...")

    create_demo = input("Create demo version? (y/n): ").lower().strip()

    if create_demo == 'y':
        print("🎨 Creating demo interface...")

        # Demo version
        import gradio as gr
        import random

        def demo_analysis(dream_text):
            if not dream_text.strip():
                return "Please enter a dream to analyze."

            # Simulate your model's analysis style
            analyses = [
                f"From a Freudian perspective, your dream about '{dream_text[:30]}...' reveals significant unconscious symbolism. The imagery suggests repressed desires seeking expression through the unconscious mind. According to psychoanalytic theory, dreams are the 'royal road to the unconscious' and often contain disguised fulfillments of suppressed wishes.",

                f"This dream contains rich psychoanalytic material. Freud would interpret the elements in '{dream_text[:30]}...' as symbolic representations of deeper psychological conflicts. The manifest content (what you remember) conceals latent content (hidden meanings) through mechanisms of displacement and condensation.",

                f"Your dream imagery reflects classic Freudian concepts. The narrative in '{dream_text[:30]}...' likely represents wish fulfillment and the unconscious mind's attempt to process repressed material. The symbolic nature of dream elements often relates to infantile desires and unresolved psychological tensions."
            ]

            analysis = random.choice(analyses)

            return f"""
            <div style='padding: 20px; background: linear-gradient(135deg, #ff6b6b 0%, #ee5a24 100%); border-radius: 15px; color: white; margin: 10px 0;'>
                <h2 style='margin-top: 0; color: #fff;'>🧠 Demo Freudian Analysis</h2>
                <div style='background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px; line-height: 1.6;'>
                    {analysis}
                </div>
                <p style='margin-bottom: 0; font-size: 0.9em; opacity: 0.8;'>
                    ⚠️ DEMO MODE - Not your actual model • Authenticate for real analysis
                </p>
            </div>
            """

        # Create demo interface
        with gr.Blocks(title="🧠 Demo Freudian Analyzer") as demo_interface:

            gr.HTML("""
            <div style='text-align: center; padding: 20px; background: linear-gradient(135deg, #ff6b6b 0%, #ee5a24 100%); border-radius: 15px; color: white; margin-bottom: 20px;'>
                <h1 style='margin: 0; font-size: 2.5em;'>🧠 Demo Freudian Analyzer</h1>
                <p style='margin: 10px 0 0 0; font-size: 1.2em; opacity: 0.9;'>
                    Demo Mode - Authenticate to Use Your Real Model
                </p>
            </div>
            """)

            gr.Markdown("""
            ### ⚠️ This is a DEMO version

            To use **YOUR actual trained model**, you need:
            1. Hugging Face authentication for Llama-2 access
            2. Your 160MB LoRA adapter requires the correct base model

            **Your Model Details:**
            - Base: `unsloth/llama-2-7b-chat-bnb-4bit`
            - LoRA: 16 rank, 32 alpha
            - Target: Llama-2 attention layers
            """)

            dream_input = gr.Textbox(
                lines=6,
                placeholder="Enter your dream (this is demo mode - not your actual model)",
                label="🌙 Dream Description"
            )

            analyze_btn = gr.Button("🔍 Demo Analysis", variant="secondary")
            analysis_output = gr.HTML()

            analyze_btn.click(
                fn=demo_analysis,
                inputs=[dream_input],
                outputs=[analysis_output]
            )

        # Launch demo
        demo_interface.launch(share=True, debug=False)

    else:
        print("❌ Cannot proceed without authentication.")

else:
    # AUTHENTICATED - Load your real model!
    print("🎉 AUTHENTICATION SUCCESS! Loading YOUR real model...")

    import torch
    from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
    from peft import PeftModel
    import gradio as gr

    # Clear GPU memory
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print(f"🔥 GPU: {torch.cuda.get_device_name()}")

    class AuthenticatedModelLoader:
        def __init__(self):
            self.model = None
            self.tokenizer = None
            self.model_loaded = False
            self.load_error = None

            try:
                self._load_authenticated_model()
            except Exception as e:
                self.load_error = str(e)
                print(f"❌ Model loading failed: {e}")

        def _load_authenticated_model(self):
            """Load your model with proper authentication"""

            model_path = "/content/model/unsloth_model"
            base_model = "meta-llama/Llama-2-7b-chat-hf"  # Standard Llama-2

            print(f"🎯 Loading YOUR authenticated model:")
            print(f"   Base: {base_model}")
            print(f"   LoRA: {model_path}")

            # Load tokenizer
            print("📝 Loading Llama-2 tokenizer...")
            self.tokenizer = AutoTokenizer.from_pretrained(base_model)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            print("✅ Tokenizer loaded")

            # Load base model
            print("🤖 Loading Llama-2 base model...")
            self.model = AutoModelForCausalLM.from_pretrained(
                base_model,
                device_map="auto",
                torch_dtype=torch.float16,
                low_cpu_mem_usage=True,
                trust_remote_code=True
            )
            print("✅ Base model loaded")

            # Load YOUR LoRA
            print("🔗 Loading YOUR LoRA adapter...")
            self.model = PeftModel.from_pretrained(self.model, model_path)
            self.model.eval()
            print("✅ YOUR LoRA loaded successfully!")

            self.model_loaded = True
            print("🎉 YOUR CUSTOM MODEL IS FULLY LOADED!")

        def analyze_dream(self, dream_text):
            if not self.model_loaded:
                return f"❌ Model failed to load: {self.load_error}"

            if not dream_text.strip():
                return "Please enter a dream to analyze."

            # Clear memory
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

            # Your model's training format
            system_prompt = "You are an expert Freudian psychoanalyst. Analyze dreams using Freudian psychological concepts, symbolism, and unconscious desires."
            prompt = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\nAnalyze this dream from a Freudian perspective:\n\n{dream_text} [/INST]"

            try:
                inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1200)
                inputs = {k: v.to(self.model.device) for k, v in inputs.items()}

                with torch.no_grad():
                    outputs = self.model.generate(
                        **inputs,
                        max_new_tokens=400,
                        do_sample=True,
                        temperature=0.7,
                        top_p=0.9,
                        repetition_penalty=1.1,
                        pad_token_id=self.tokenizer.eos_token_id,
                        eos_token_id=self.tokenizer.eos_token_id
                    )

                response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                analysis = response[len(prompt):].strip()
                analysis = analysis.replace("[/INST]", "").replace("</s>", "").strip()

                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

                return f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); border-radius: 15px; color: white; margin: 10px 0;'>
                    <h2 style='margin-top: 0; color: #fff;'>🧠 YOUR Custom Freudian Analysis</h2>
                    <div style='background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px; line-height: 1.6;'>
                        {analysis}
                    </div>
                    <p style='margin-bottom: 0; font-size: 0.9em; opacity: 0.8;'>
                        🎯 Generated by YOUR Custom LoRA (16/32) • Specialized for Freudian Analysis
                    </p>
                </div>
                """

            except Exception as e:
                return f"❌ Generation error: {str(e)}"

    # Create interface for your authenticated model
    print("🎨 Creating interface for YOUR authenticated model...")

    analyzer = AuthenticatedModelLoader()

    with gr.Blocks(title="🧠 YOUR Authenticated Model") as interface:

        gr.HTML("""
        <div style='text-align: center; padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); border-radius: 15px; color: white; margin-bottom: 20px;'>
            <h1 style='margin: 0; font-size: 2.5em;'>🧠 YOUR Custom Freudian Model</h1>
            <p style='margin: 10px 0 0 0; font-size: 1.2em; opacity: 0.9;'>
                ✅ Authenticated & Fully Loaded
            </p>
        </div>
        """)

        status = "🎯 YOUR Model Active" if analyzer.model_loaded else "❌ Loading Failed"

        gr.Markdown(f"""
        ### 🔥 GPU Active • {status}

        **Congratulations!** YOUR custom model is now properly loaded:
        - ✅ Authenticated access to Llama-2
        - ✅ YOUR 160MB LoRA adapter loaded
        - ✅ Correct target modules matched
        - ✅ Ready for specialized Freudian analysis
        """)

        dream_input = gr.Textbox(
            lines=7,
            placeholder="Describe your dream... YOUR custom model will provide specialized Freudian analysis.",
            label="🌙 Dream Description"
        )

        with gr.Row():
            analyze_btn = gr.Button("🔍 Analyze with YOUR Model", variant="primary", size="lg")
            clear_btn = gr.Button("🗑️ Clear")

        analysis_output = gr.HTML(label="YOUR Model's Analysis")

        # Examples
        examples = [
            ["I was flying over water but my wings turned heavy and I fell toward dark waves below."],
            ["I found myself in my childhood home but all the rooms were rearranged and unfamiliar."],
            ["I was being chased through a maze by a shadowy figure I couldn't identify."],
            ["I was taking an exam but the questions were all about my personal secrets."],
            ["I was at a party where I knew everyone but realized I wasn't wearing clothes."]
        ]

        gr.Examples(examples, inputs=dream_input, label="💭 Test YOUR Model")

        def clear_all():
            return "", ""

        analyze_btn.click(
            fn=analyzer.analyze_dream,
            inputs=[dream_input],
            outputs=[analysis_output]
        )

        clear_btn.click(
            fn=clear_all,
            outputs=[dream_input, analysis_output]
        )

        gr.HTML("""
        <div style='text-align: center; padding: 15px; margin-top: 20px; border-top: 1px solid #eee; color: #666;'>
            <p>🎯 YOUR Custom Model • Llama-2 + 160MB LoRA • Freudian Specialist</p>
        </div>
        """)

    # Launch your authenticated model
    interface.launch(share=True, debug=False)
    print("🎉 YOUR AUTHENTICATED MODEL IS NOW RUNNING!")

print("\n" + "="*50)
print("SUMMARY:")
if authenticated:
    print("✅ SUCCESS: Your model should now be working with proper Llama-2 base!")
else:
    print("⚠️ DEMO: Authenticate with Hugging Face to use your real model")
print("="*50)

🔑 SETTING UP HUGGING FACE AUTHENTICATION

🎯 YOUR LORA NEEDS LLAMA-2 BASE MODEL!

The issue: Your LoRA was trained on Llama-2 architecture, but we can't access it without authentication.

SOLUTION: Get authenticated access to Llama-2 models

📋 STEP-BY-STEP INSTRUCTIONS:

1. 🌐 Go to: https://huggingface.co/meta-llama/Llama-2-7b-chat-hf
2. 📝 Create a Hugging Face account (if you don't have one)
3. ✅ Request access to Llama-2 (usually approved quickly)
4. 🔑 Create an access token:
   - Go to: https://huggingface.co/settings/tokens
   - Click "New token"
   - Choose "Read" permissions
   - Copy the token

5. 🔐 Add token to Colab:
   - Click the key icon (🔑) in the left sidebar
   - Add new secret: Name = "HF_TOKEN", Value = your token
   - Enable notebook access

OR run this code with your token:

🔑 AUTHENTICATION OPTIONS:

Option 1 - Add HF_TOKEN to Colab secrets (RECOMMENDED)
Option 2 - Login directly (run the code below)
⚠️ No HF_TOKEN in secrets: Secret HF_TOKEN does not exist.

🔑 MANUA

✅ Authentication successful!
🎉 AUTHENTICATION SUCCESS! Loading YOUR real model...
🔥 GPU: Tesla T4
🎨 Creating interface for YOUR authenticated model...
🎯 Loading YOUR authenticated model:
   Base: meta-llama/Llama-2-7b-chat-hf
   LoRA: /content/model/unsloth_model
📝 Loading Llama-2 tokenizer...
❌ Model loading failed: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
401 Client Error. (Request ID: Root=1-683c85d9-489a5429488131f21dffcbc4;5a9e655d-1230-45e2-9747-48ce2a770bd3)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted. You must have access to it and be authenticated to access it. Please log in.
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2ce0b0cc6664d58f6b.gradio.live

This share link expires in 72 hours. For fre

AttributeError: module 'gradio' has no attribute 'blocks'